#Install and Import libraries

In [ ]:
!git clone https://github.com/amontgomerie/question_generator
%cd question_generator/
!git reset --hard   50982ff2e6dffc752b7debc938126ff51a61278a 
!python -m pip install -e .
%cd ..

In [ ]:
!pip install --upgrade git+https://github.com/ramsrigouthamg/Questgen.ai
!pip install --quiet git+https://github.com/boudinfl/pke.git@69337af9f9e72a25af6d7991eaa9869f1322dd72

In [ ]:
!python -m nltk.downloader universal_tagset
!python -m spacy download en

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz

# Restart Runtime

We need to restart runtime after installing the required libraries by running the following command

In [ ]:
import os
os.kill(os.getpid(), 9)

# Now we import our libraries

In [ ]:
import nltk,warnings,os,contextlib
nltk.download('stopwords')
warnings.filterwarnings('ignore')
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_colwidth', 0)
import ipywidgets as widgets
import numpy as np
from questiongenerator import QuestionGenerator
from Questgen import main
qgt5 = QuestionGenerator()
qg = main.QGen()

#Define main functions

In [ ]:
def getContext(paras,maxlen=500,minlen=10):
  context=[]
  i=0
  while i<len(paras):
    lp=len(nltk.word_tokenize(paras[i]))
    if not paras[i] or lp<minlen or paras[i].strip()[-1]!='.':
      i+=1
    elif lp<maxlen:
      context.append(paras[i])
      i+=1
    else:
      sents=nltk.sent_tokenize(paras[i])
      sentences=[]
      l=0
      j=0
      while j<len(sents):
        ll=len(nltk.word_tokenize(sents[j]))
        if ll+l<maxlen:
          sentences.append(sents[j])
          l+=ll
        else:
          context.append(' '.join(sentences))
          sentences=[]
          l=0
          sentences.append(sents[j-1])
          sentences.append(sents[j])
          l+=len(nltk.word_tokenize(sents[j-1]))
          l+=ll
        j+=1
      if sentences:
        context.append(' '.join(sentences))
      i+=1
  return context

In [ ]:
def getMFQ(context,force=False):
  li=[]
  totalq=0
  for c in tqdm(context):
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
      output=qg.predict_mcq({"input_text": c})
      if 'questions' in output:
        for q in output['questions']:
          totalq+=1
          li.append([c,q['question_statement'],q['answer'],'Multichoice'])
      output=qg.predict_shortq({"input_text": c})
      if 'questions' in output:
        for q in output['questions']:
          totalq+=1
          li.append([c,q['Question'],q['Answer'],'Multichoice'])
      output=qgt5.generate(c,answer_style='multiple_choice',use_evaluator=False)
      for q in output:
        for a in q['answer']:
          if a['correct']:
            totalq+=1
            li.append([c,q['question'],a['answer'],'Multichoice'])
      output=qgt5.generate(c,answer_style='sentences',use_evaluator=False)
      for q in output:
          a=q['answer']
          totalq+=1
          if force:
            for s in nltk.sent_tokenize(c):
              if a.replace(' ','') in s.replace(' ',''):
                li.append([c,q['question'],s,'Full Sentence'])
                break
          else:
            li.append([c,q['question'],a,'Full Sentence'])
  print("### MFQ",totalq)
  MCQFQ=pd.DataFrame(li,columns=["Context","Question","Answer","Type"])
  return MCQFQ

#Run 

You can provide the path to a text file as **filename** variable

Or input the text directly as a string in the **text** variable

change the boolean variable **isString** according to your choice

In [ ]:
text="""
Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.\n
The Lunar Exploration Light Rover shall accelerate back up to 15 km/h within ten seconds of stopping, and initiate each stop within ten seconds of reaching 15 km/h. For each of the five stops, the Lunar Exploration Light Rover shall stop within a distance of 4.2 m. Obstacle Crossing: The Lunar Exploration Light Rover is required to drive at low speed over obstacles of up to 0.3 m wide, 0.3 m long, and up to 0.3 m high with edges rounded to a radius of 0.05m. The Lunar Exploration Light Rover needs to be able to pass over an obstacle with a height of 0.35 m and a width of 0.7 m. No projection of the Lunar Exploration Light Rover shall be closer to the ground than 0.35 m, other than the wheels, suspension and drivetrain components. Maximum Gradient: The Lunar Exploration Light Rover shall be able to start and stop in a controlled fashion while ascending or descending a 25° slope at maximum gross vehicle weight. Performance: There shall be no stalling, slipping, overheating, upsetting or hesitation.
"""

In [ ]:
isString=True
if isString:
  doc=text.split('\n')
else:
  filename="path/to/file.txt"
  tf=open(filename,"r")
  doc=tf.read().split('\n')
  tf.close() 

Now we split the text into contexts and generate Q-A pairs

In [ ]:
contexts=getContext(doc,maxlen=125)
qaPairs=getMFQ(contexts)

100%|██████████| 4/4 [00:43<00:00, 10.78s/it]

### MFQ 85


In [ ]:
qaPairs

,Context,Question,Answer,Type
0,"Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.",What is the easiest way to embark on an EVA mission?,disembark,Multichoice
1,"Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.",What is the easiest way to embark on an EVA mission?,disembark,Multichoice
2,"Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.",what is the crew transportation?,Crew Transportation: Nominal:,Multichoice
3,"Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.",How many EVA-suited crew members can the Lunar Exploration Light Rover transport?,one,Multichoice
4,"Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member. EVA Embarcation: When upgrade to transport astronaut, the Lunar Exploration Light Rover shall provide convenient access for EVA crew to embark and disembark. Cargo Transportation: The Lunar Exploration Light Rover shall transport cargo items.",how many crew members are on the lvl?,Crew Transportation: Nominal: The Lunar Exploration Light Rover needs to be designed in a way that is upgradeable to transport one EVA-suited crew member.,Full Sentence
...,...,...,...,...
80,"The Lunar Exploration Light Rover needs to be able to pass over an obstacle with a height of 0.35 m and a width of 0.7 m. No projection of the Lunar Exploration Light Rover shall be closer to the ground than 0.35 m, other than the wheels, suspension and drivetrain components. Maximum Gradient: The Lunar Exploration Light Rover shall be able to start and stop in a controlled fashion while ascending or descending a 25° slope at maximum gross vehicle weight. Performance: There shall be no stalling, slipping, overheating, upsetting or hesitation.",what is the height of the lunar exploration light rover?,No projection of the Lunar Exploration Light Rover shall be closer to the ground than 0.,Full Sentence
81,"The Lunar Exploration Light Rover needs to be able to pass over an obstacle with a height of 0.35 m and a width of 0.7 m. No projection of the Lunar Exploration Light Rover shall be closer to the ground than 0.35 m, other than the wheels, suspension and drivetrain components. Maximum Gradient: The Lunar Exploration Light Rover shall be able to start and stop in a controlled fashion while ascending or descending a 25° slope at maximum gross vehicle weight. Performance: There shall be no stalling, slipping, overheating, upsetting or hesitation.",what is the london exploration light rover?,Maximum Gradient: The Lunar Exploration Light Rover shall be able to start and stop in a controlle

You can export the results as an excel file or as csv using **to_excel** or **to_csv** command

In [ ]:
qaPairs.to_excel('generatedQA.xlsx')